In [7]:
import datetime
import requests
import json
import pandas as pd
import os

from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
from pathlib import Path
from src.utils import get_root_dir

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Setup

In [9]:
root_dir = get_root_dir()
data_dir = root_dir / "data" / "deployment" / "raw"
filepath = data_dir / "consumption.csv"

In [10]:
load_dotenv()
epias_username = os.getenv("epias_username")
epias_password = os.getenv("epias_password")

In [11]:
years_of_data = 1
consumption_lag = 2

In [12]:
current_date = datetime.datetime.now().astimezone().replace(microsecond = 0)
current_date.isoformat()

'2024-09-18T16:02:46+03:00'

In [13]:
type(current_date.isoformat())

str

In [14]:
end_date = current_date - datetime.timedelta(hours = 2)
end_date.isoformat()

'2024-09-18T14:02:46+03:00'

In [15]:
start_date = end_date - relativedelta(years = years_of_data)
start_date.isoformat()

'2023-09-18T14:02:46+03:00'

## TGT request

https://seffaflik.epias.com.tr/electricity-service/technical/en/index.html#_adding_security_information_to_requests

In [16]:
login_url = "https://giris.epias.com.tr/cas/v1/tickets"

In [17]:
login_headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "text/plain"
}

In [18]:
login_body = {
    "username": epias_username,
    "password": epias_password
}

In [19]:
tgt_request = requests.post(
    login_url,
    data = login_body,
    headers = login_headers
)

In [23]:
tgt_request.ok

True

In [41]:
tgt_request.status_code

201

In [24]:
tgt = tgt_request.text

TGT is returned as plaintext.

## Real Time Consumption Data Listing Service

https://seffaflik.epias.com.tr/electricity-service/technical/tr/index.html#_realtime-consumption

Endpoint returns 1 year of data maximum.

In [25]:
base_url = "https://seffaflik.epias.com.tr/electricity-service"

In [26]:
consumption_url = base_url + "/v1/consumption/data/realtime-consumption"

In [27]:
consumption_url

'https://seffaflik.epias.com.tr/electricity-service/v1/consumption/data/realtime-consumption'

In [28]:
consumption_body = {
    "startDate": start_date.isoformat(),
    "endDate": end_date.isoformat()
}

In [29]:
consumption_headers = {
    "Accept-Language": "en",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "TGT": tgt,
}

In [30]:
consumption_response = requests.post(
    consumption_url,
    data = json.dumps(consumption_body),
    #data = consumption_body,
    headers = consumption_headers
)

In [32]:
consumption_response.status_code

200

In [33]:
consumption_response.ok

True

In [34]:
response_data = json.loads(consumption_response.text)

In [35]:
# "statistics" holds min-max-avg-total values
response_data["statistics"]

{'consumptionTotal': 338319316.7,
 'consumptionAvg': 38554.91,
 'consumptionMin': 20062.44,
 'consumptionMax': 57772.4}

In [36]:
# "items" holds the time series data
df = pd.DataFrame(response_data["items"])

In [37]:
df

,date,time,consumption
0,2023-09-19T00:00:00+03:00,00:00,35876.29
1,2023-09-19T01:00:00+03:00,01:00,34437.03
2,2023-09-19T02:00:00+03:00,02:00,33390.71
3,2023-09-19T03:00:00+03:00,03:00,32695.29
4,2023-09-19T04:00:00+03:00,04:00,32319.74
...,...,...,...
8770,2024-09-18T10:00:00+03:00,10:00,44244.60
8771,2024-09-18T11:00:00+03:00,11:00,45237.84
8772,2024-09-18T12:00:00+03:00,12:00,43580.20
8773,2024-09-18T13:00:00+03:00,13:00,44611.92


End date is sometimes rounded up to the next hour: 
- Request date: 11:59am,
- end_date: 9:59am,
- Final date in returned data: 10:00am,

Not a problem, as the data for 10-11am is nearly complete by 11:59am.

In [38]:
df.to_csv(
    filepath,
    index = False
)

## Real Time Consumption Export Service

https://seffaflik.epias.com.tr/electricity-service/technical/en/index.html#_realtime-consumption-export

In [29]:
export_url = base_url + "/v1/consumption/export/realtime-consumption"

In [30]:
export_url

'https://seffaflik.epias.com.tr/electricity-service/v1/consumption/export/realtime-consumption'

In [31]:
export_body = {
    "startDate": start_date.isoformat(),
    "exportType": "CSV",
    "endDate": end_date.isoformat()
}

This endpoint also requires a TGT in the request body, **even though it's not listed in the documentation**.

In [32]:
export_headers = {
    "Accept-Language": "en",
    "Accept": "application/json",
    "Content-Type": "application/json",
    "TGT": tgt,
}

In [33]:
export_response = requests.post(
    export_url,
    data = json.dumps(export_body),
    #data = export_body,
    headers = export_headers
)

In [34]:
export_response

<Response [200]>

This endpoint seems to yield successful responses with <=1 year requests, but in HTML format. The code below can be used to print & save it. Could also be exported to .csv.

It yields an error with with >1 year requests, same as the first endpoint. But the "Content-Type" changes to JSON.